In [1]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import cv2, os, math, time
from datetime import timedelta
from sklearn.utils import shuffle
from numpy.lib.stride_tricks import as_strided
from itertools import product
from functools import reduce

In [2]:
###Configuration
"""
Data Configurations/Paths
"""
img_dir="../../original_images/SD"
img_lbls=""
base_model = 'SD/sd_01_.ckpt'
model_20000 = 'SD/sd_20000.ckpt'
model_30000 = 'SD/sd_30000.ckpt'
model_40000 = 'SD/sd_40000.ckpt'
model_50000 = 'SD/sd_50000.ckpt'
model2_50000 = 'SD/sd2_50000.ckpt' #this model takes a 10x10 predicted mask from model:1 and predicts an image which combines all the patches.

connected_model = 'SD/sd_conected.ckpt'

##
# Convolutional Layer 1.
filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
num_filters1 = 16         # There are 16 of these filters.

# filter_size1 = 16          # Convolution filters are 4 x 4 pixels.
# num_filters1 = 64         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 8          # Convolution filters are 2 x 2 pixels.
num_filters2 = 64         # There are 32 of these filters.

# Convolutional Layer 3.
filter_size3 = 8          # Convolution filters are 2 x 2 pixels.
num_filters3 = 64         # There are 64 of these filters.

# Convolutional Layer 4.
filter_size4 = 4          # Convolution filters are 2 x 2 pixels.
num_filters4 = 32         # There are 128 of these filters.

# Fully-connected layer.
fc_size = 2000             # Number of neurons in fully-connected layer.

# We know that images are 60 pixels in each dimension.
# img_size = 8 * 4

# Images are stored in one-dimensional arrays of this length.
img_size_flat = 10 * 10

# Number of colour channels for the images: 3 channel for RGB.
num_channels = 1

# Tuple with height and width of images used to reshape arrays.
img_shape = (10, 10, num_channels)

# Number of classes, one class for same or different image
num_classes = 2*2
patch_size = (2, 2, 3)
npatches = 2

In [3]:
def load_data(img_dir):
        list_of_orig_imgs = []
        list_of_pred_imgs = []
        list_of_labels = []
        list_same_diff = []
        list_img_keys = []
        for img in os.listdir(img_dir):
            
            img_path = os.path.join(img_dir, img)
            list_same_diff.append(int(os.listdir(img_path)[0]))
            list_img_keys.append(img)
            img_path = img_path + "/" + os.listdir(img_path)[0]
            for img_label in os.listdir(img_path):
                img_data = os.path.join(img_path, img_label)
                if img_label == "img":
#                     print(img_data + "/img.png")
                    list_of_orig_imgs.append(img_data + "/img.png")
                    list_of_pred_imgs.append(img_data + "/predicted_mask.png")
                else:
                    list_of_labels.append([os.path.join(img_data, label) for label in os.listdir(img_data)])

        data_imgs = np.array([list_of_orig_imgs, list_of_pred_imgs])
        data_labels = np.array(list_of_labels)
        data_same_diff = np.array(list_same_diff)
        data_img_keys = np.array(list_img_keys)

        return data_imgs, data_labels, data_same_diff, data_img_keys

    
def get_batch_images(data_orig, data_pred, label, same_diff, img_keys, rshp, grey_scale):
        list_of_orig_imgs = []
        list_of_pred_imgs = []
        list_of_labels = []
        list_of_same_diff = []
        list_of_img_keys = []
        for img_orig, img_pred, lbl, img_type, img_key in zip(data_orig, data_pred, label, same_diff, img_keys):
            if (grey_scale):
                orig_img = cv2.imread(img_orig)
                predicted_msk = cv2.imread(img_pred, cv2.IMREAD_GRAYSCALE)
            else:
                orig_img = cv2.imread(img[0])
                
            orig_lbl = cv2.imread(lbl)
            if orig_img is None or orig_lbl is None:
                    print ("Unable to read image{} or {}".format(img[0], lbl))
                    continue
            
            if (grey_scale):
                orig_lbl = rgb2grey(orig_lbl)

            flattened_orig_img = orig_img.flatten()
            flattened_pred_img = predicted_msk.flatten()
            flattened_lbl = orig_lbl.flatten()
            
            if grey_scale:
                flattened_lbl = np.reshape(flattened_lbl, [10, 10])
#                 print(flattened_lbl)
#                 flattened_lbl = normalize(flattened_lbl)
#                 print(flattened_lbl)
            
            list_of_orig_imgs.append(np.asarray(flattened_orig_img, dtype=np.float32))
            list_of_pred_imgs.append(np.asarray(flattened_pred_img, dtype=np.float32))
        
            list_of_labels.append(np.asarray(flattened_lbl, dtype=np.float32))
            list_of_same_diff.append(img_type)
            list_of_img_keys.append(img_key)

        data_labels = np.array(list_of_labels)
#         print(data_labels.shape)
#         print(rshp)
#         reshaped_labels = np.reshape(data_labels, rshp)
#         data_labels = np.squeeze(reshaped_labels[:, :, :1])
        data_imgs = np.array([list_of_orig_imgs, list_of_pred_imgs])
        data_img_type = np.array(list_of_same_diff)
        data_img_keys = np.array(list_of_img_keys)
        
        """this function call locates top left location of each patch in the mask and return. Comment it if contents are required."""
#         print(data_labels.shape)
#         data_labels = get_patch_loc(data_labels)
        
        return data_imgs, data_labels, data_img_type, data_img_keys

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data[0]))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_orig = data[0, :]
    data_pred = data[1, :]
    data_orig_shuffle = [data[0, i] for i in idx]
    data_pred_shuffle = [data[1, i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_orig_shuffle), np.asarray(data_pred_shuffle), np.asarray(labels_shuffle)

def rgb2grey(rgb):
    return(np.dot(rgb[...,:3], [0.299, 0.587, 0.114]))

def get_patch_loc(imgs):
    img_loc = []
    for img in imgs:
        img = np.squeeze(img)
        img[np.where(img == 255)] = 1.
        r = img_shape[0] - patch_size[0] + 1
        c = img_shape[1] - patch_size[0] + 1

        iter_o = np.array(list(product(range(0,patch_size[0]), range(0,patch_size[1]))))
        left_corners = reduce(lambda x,y: np.multiply(x,y), map(lambda x, y: img[x:x+r, y:y+c], iter_o[:, 0], iter_o[:, 1]))
        left_top_locations = np.transpose(np.where(left_corners == 1))
        left_top_locations = np.array(left_top_locations, dtype=np.int32)
        img_loc.append(left_top_locations)
        
    return np.array(img_loc)

In [50]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)

train_org = train_data[0, 0:2]
train_prd = train_data[1, 0:2]

merged_patch_lbls = train_labels[0:2, 0]
mask_lbls = train_labels[0:2, 1]
img_type_lbl = img_type[0:2]
img_key = img_keys[0:2]
dims = (2, 100, num_channels)
# print(train, merged_patch_lbls, mask_lbls)
train, labels, img_type_lbl, img_key = get_batch_images(train_org, train_prd, mask_lbls, img_type_lbl, img_key, dims, True)

(2, 10, 10)


In [62]:
print(labels)

[[[4 1]
  [4 8]]

 [[1 3]
  [7 8]]]


In [4]:
def new_weights(shape, layer_name):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.Variable(initializer(shape), name=layer_name+'_W')

def new_bias(length, layer_name):
    return tf.Variable(tf.constant(0.05, shape=[length]), name=layer_name+'_b')

def new_conv_layer(input,
                   num_input_channels,
                   filter_size,
                   num_filters,
                   name_scope,
                   layer_name='',
                   use_pooling=True):

    with tf.name_scope(name_scope):
        shape = [filter_size, filter_size, num_input_channels, num_filters]
        weights = new_weights(shape, layer_name)
        biases = new_bias(num_filters, layer_name)

        layer = tf.add(tf.nn.conv2d(input=input, filter=weights, strides=[1,1,1,1], padding='SAME'), biases, name=layer_name)

        if use_pooling:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 3, 3, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME', name=layer_name+'_max')
            
        layer = tf.nn.relu(layer, name=layer_name+'_activation')

    return layer, weights

def flatten_layer(layer, flag=False):
    layer_shape = layer.get_shape()
    if flag is True:
        num_features = layer_shape.num_elements()
        layer_flat = tf.reshape(layer, [-1, num_features])
    else:
        num_features = layer_shape[1:4].num_elements()
        layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features

def new_fc_layer(input,
                num_inputs,
                num_outputs,
                name_scope,
                layer_name='',
                use_relu=True):
    
    with tf.name_scope(name_scope):
        weights = new_weights([num_inputs, num_outputs], layer_name)
        biases = new_bias(num_outputs, layer_name)

        layer = tf.add(tf.matmul(input, weights),biases,name=layer_name)
    #     layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer, layer_name+'_activation')
    
    return layer


def normalise_tensor(tensor):
    return tf.div(
   tf.subtract(
      tensor, 
      tf.reduce_min(tensor)
   ), 
   tf.subtract(
      tf.reduce_max(tensor), 
      tf.reduce_min(tensor)
   )
)

In [8]:
x_p = tf.placeholder(tf.float32, shape=[None, img_size_flat*num_channels], name='x_p')
x_o = tf.placeholder(tf.float32, shape=[None, img_size_flat*3], name='x_o')
x_pred_image = tf.reshape(x_p, [-1, 10, 10, num_channels])
x_orig_image = tf.reshape(x_p, [-1, 10, 10, 3])
k = tf.constant(npatches)
y_true = tf.placeholder(tf.int32, shape=[None, 2, 2], name='y_true')
y_true_cls = tf.placeholder(tf.int32, shape=[None, 2, 2], name='y_true_cls')


In [12]:
def new_custom_layer(input,
                     orig_img,
                     k_,
                num_inputs,
                num_outputs,
                name_scope,
                layer_name='',
                use_relu=True):
    
    with tf.name_scope(name_scope):
        resh_inp = tf.reshape(input, [-1, 10, 10])
        input_shape = tf.shape(resh_inp)
        rows, cols = input_shape[1], input_shape[2]
        d_rows, d_cols = 2, 2
        subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1
# #         # Index grids
        ii, jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')
        d_ii, d_jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
        
#         # Add indices
        subm_ii = ii[:, :, tf.newaxis, tf.newaxis] + d_ii
        subm_jj = jj[:, :, tf.newaxis, tf.newaxis] + d_jj
#         # Make submatrices tensor
        subm = tf.gather_nd(resh_inp[0, :, :], tf.stack([subm_ii, subm_jj], axis=-1), name=layer_name + "_gather")
#         # Add submatrices
        subm_sum = tf.reduce_sum(subm, axis=(2, 3), name=layer_name + "_subm_sum")
#         # Use TopK to find top submatrices
        _, top_idx = tf.nn.top_k(tf.reshape(subm_sum, [-1]), tf.minimum(k, tf.size(subm_sum)), name=layer_name + "_top_idx")
#         # Get row and column
        top_row = top_idx // subm_cols
        top_col = top_idx % subm_cols
        result = tf.stack([top_row, top_col], axis=-1, name=layer_name + "_result")
        result_shape = tf.shape(result)
        result = tf.reshape(result, [-1, result_shape[0], result_shape[1]], name=layer_name + "_shaped_result")
# Working Mask
#         patches = tf.map_fn(lambda x: tf.cast(resh_inp[:, x[0]:x[0] + patch_size[0], x[1]:x[1] + patch_size[1]], dtype=tf.float32), result, dtype=tf.float32)
#         patch_shape = tf.shape(patches)
#         comb_patches = tf.reshape(patches, [-1, patch_shape[1], patch_shape[2]* patch_shape[3], patch_shape[3]], name=layer_name + "_comb_patches")

#         patches = tf.squeeze(tf.map_fn(lambda x: tf.cast(resh_inp[:, x[0]:x[0] + patch_size[0], x[1]:x[1] + patch_size[1]], dtype=tf.float32), result, dtype=tf.float32))
#         patch_shape = tf.shape(patches)
#         comb_patches = tf.reshape(patches, [-1, patch_shape[0] * patch_shape[1], patch_shape[2], patch_shape[3]], name=layer_name + "_comb_patches")


#Not working 
#         patches = tf.squeeze(tf.map_fn(lambda x: tf.cast(orig_img[:, x[0]:x[0] + patch_size[0], x[1]:x[1] + patch_size[1], :], dtype=tf.float32), result, dtype=tf.float32))
#         patch_shape = tf.shape(patches)
#         comb_patches = tf.reshape(patches, [-1, patch_shape[0] * patch_shape[1], patch_shape[2], patch_shape[3]], name=layer_name + "_comb_patches")
        
    return result

In [13]:
layer1_conv1, weights_conv1 = new_conv_layer(input=x_pred_image,
                                            num_input_channels=num_channels,
                                            filter_size=filter_size1,
                                            num_filters=num_filters1,
                                             name_scope = 'cv',
                                             layer_name='conv1',
                                            use_pooling=True)

# layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
#                                            num_input_channels=num_filters1,
#                                            filter_size=filter_size2,
#                                            num_filters=num_filters2,
#                                              name_scope = 'cv',
#                                              layer_name='conv2',
#                                            use_pooling=True)

# layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
#                                            num_input_channels=num_filters2,
#                                            filter_size=filter_size3,
#                                            num_filters=num_filters3,
#                                              name_scope = 'cv',
#                                              layer_name='conv3',
#                                            use_pooling=True)

# layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
#                                            num_input_channels=num_filters3,
#                                            filter_size=filter_size4,
#                                            num_filters=num_filters4,
#                                              name_scope = 'cv',
#                                              layer_name='conv4',
#                                            use_pooling=True)


layer_flat, num_features = flatten_layer(layer1_conv1)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=img_size_flat,
                         name_scope = 'fc',
                         layer_name = 'fc_1',
                         use_relu=False)


# layer_fc2 = normalise_tensor(layer_fc1)

# layer_fc3 = new_fc_layer(input=layer_fc2,
#                          num_inputs=num_classes,
#                          num_outputs=num_classes,
#                          name_scope = 'fc',
#                          layer_name = 'fc3',
#                          use_relu=False)

layer_fc3 = new_custom_layer(input=layer_fc1,
                             orig_img = x_orig_image,
                             k_=k,
                         num_inputs=img_size_flat,
                         num_outputs=num_classes,
                         name_scope = 'custom',
                         layer_name = 'custom_1',
                         use_relu=False)

# layer_fc3 = new_fc_layer(input=layer_fc2,
#                          num_inputs=num_classes,
#                          num_outputs=num_classes,
#                          name_scope = 'fc',
#                          layer_name = 'fc2',
#                          use_relu=False)


# print(layer_fc2)
y_pred_cls = layer_fc3
print(y_pred_cls)

Tensor("custom/custom_1_shaped_result:0", shape=(?, ?, ?), dtype=int32)


In [14]:
cost = tf.reduce_mean(tf.square(y_true - y_pred_cls))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# ## some more performance measures
# correct_prediction = tf.equal(y_pred_cls, y_true)
# accuracy, acc_op = tf.metrics.accuracy(labels=tf.argmax(y_true, axis=1), predictions=tf.argmax(y_pred_cls, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

ValueError: Invalid type tf.int32 for Mean:0, expected: [tf.float32, tf.float64, tf.float16, tf.bfloat16].

In [ ]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)
session = tf.Session()
init = tf.global_variables_initializer()
session.run(init)
train_labels = train_labels[:][:, 0]
train_orig_data = train_data[0, :]
train_pred_data = train_data[1, :]
img_type = img_type[:]
img_keys = img_keys[:]
total_imgs = len(img_type)
train_batch_size = 64

def optimize(num_epochs, save_model=True,save_name= "base_model",restore_model=False,restore_name=None):
    total_iterations = 0
    done_train_imgs = 0
    start_time = time.time()
    start_ = 0
    end_ = train_batch_size    
    plot_accuracy=[]
    plot_accuracy_epoch=[]
    plot_training_size=[]
    plot_training_size_epoch=[]
    saver = tf.train.Saver()
    sum_accuracy = 0.0
    n = 1
    
        #to save the model
    for i in range(0, num_epochs):   
        start_batch=0
        end_batch = train_batch_size
        
        print("Epoch:", i + 1)
        
        if restore_model==True:
            if restore_name==None:
                print("No model file specified")
                return
            else:
                saver.restore(session,restore_name)
        
        sum_accuracy = 0.0
        n = 1
        while end_batch < total_imgs:
            train_orig = train_orig_data[start_batch:end_batch]
            train_pred = train_pred_data[start_batch:end_batch]
            labels = train_labels[start_batch:end_batch]
            img_type_lbl = img_type[start_:end_]
            img_key = img_keys[start_:end_]
            dims = (len(train_orig), num_classes, num_channels)
            train, labels, img_type_lbl, img_key = get_batch_images(train_orig, train_pred, labels, img_type_lbl, img_key, dims, True)
            if not len(train) and not len(labels):
                print("All images have been processed.")
                break;

            x_orig_batch, x_pred_batch, y_true_batch = next_batch(len(train[0]), train, labels)
            feed_dict_train = {x_o: x_orig_batch, x_p: x_pred_batch,
                       y_true: y_true_batch}
            
            session.run(optimizer, feed_dict=feed_dict_train)
    
            acc,co = session.run([accuracy, cost], feed_dict=feed_dict_train)
            sum_accuracy += acc
            n+=1
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}, Loss: {2:>.4f}"
            print(msg.format(end_batch + 1, acc, co))
            if i == num_epochs - 1:
                plot_accuracy.append(acc)
                plot_training_size.append(end_batch + 1)

            start_batch += train_batch_size
            end_batch += train_batch_size
    
        if save_model==True:
            if save_name==None:
                print("No model specified, model not being saved")
                return
            else:
                save_path = saver.save(session, save_name)
                restore_model = True
                print("Model saved in file: %s" % save_name)
        plot_accuracy_epoch.append(sum_accuracy/n)
        plot_training_size_epoch.append(i + 1)
    
    end_time = time.time()
    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))  
    print(plot_accuracy)
    print(plot_training_size)
    print(plot_accuracy_epoch)
    print(plot_training_size_epoch)

In [ ]:
save_model = True
save_name = model2_50000
restore_model=False
restore_name=model2_50000
# init = tf.global_variables_initializer()
# session.run(init)

optimize(2, save_model=True,save_name=model2_50000,restore_model=False,restore_name=model2_50000)

In [ ]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)
train = train_data[0:1]
merged_patch_lbls = train_labels[0:1, 0]
mask_lbls = train_labels[0:1, 1]
img_type_lbl = img_type[0:1]
img_key = img_keys[0:1]
dims = (1, num_classes, num_channels)
# print(train, merged_patch_lbls, mask_lbls)
train, labels, img_type_lbl, img_key = get_batch_images(train, merged_patch_lbls, img_type_lbl, img_key, dims, True)

In [ ]:
def restore_see_layer(orig, pred, model_name=None, var_name=None):
    with tf.Session('', tf.Graph()) as s:
        with s.graph.as_default():
            if ((model_name != None) and var_name != None):
                saver = tf.train.import_meta_graph(model_name+".meta")
                saver.restore(s, model_name)
#                 print(pred.shape)
                fd = {'x_orig:0': orig, 'x_pred:0':pred}
#                 print(fd.shape)
                var_name=var_name+":0"
                
                
                
                result = 0
                result = s.run(var_name, feed_dict=fd)
    return result

def see_output(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:,:]
    fig = plt.figure(figsize=figsize)
    if cmap == "gray":
        plt.imshow(img_x, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()
    
def save_patch_images(img_x1, lbl_x1, index):
    if not os.path.exists('./SD/predicted_patches/' + str(index)):
        os.makedirs('./SD/predicted_patches/' + str(index))
        os.makedirs('./SD/predicted_patches/' + str(index) + "/" + str(lbl_x1))
        
    plt.imsave('./SD/predicted_patches/' + str(index) + "/" + str(lbl_x1) + '/img.png', np.squeeze(img_x1))
    

def predict_nd_save(train, labels, img_type_lbl, img_key, start_idx):

    for index in range(0, len(train)):
        img_x = train[index:index+1, :]
        lbl_x = labels[index:index+1, :]
        img_type_x = img_type_lbl[index]
        img_key_x = img_key[index]
        prediction = restore_see_layer(ix=img_x,model_name=model2_50000,var_name='custom/custom_1_comb_patches')
        prediction = np.reshape(prediction, (1, 4, 2, 1))   
        save_patch_images(prediction, img_type_x, img_key_x)

In [ ]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)
train_mask_labels = train_labels[:][:, 1]
train_patch_labels = train_labels[:][:, 0]

batch_s = 64
total_iterations = 0
start_ = 0
end_ = batch_s
np.set_printoptions(suppress=True)

if not os.path.exists('./SD/predicted_patches'):
    os.makedirs('./SD/predicted_patches')

while True:
    train = train_data[start_:end_]
    labels = train_patch_labels[start_:end_]
    img_type_lbl = img_type[start_:end_]
    img_key = img_keys[start_:end_]
    dims = (batch_s, num_classes, num_channels)
    train, labels, img_type_lbl, img_key = get_batch_images(train, labels, img_type_lbl, img_key, dims, True)
    predict_nd_save(train, labels, img_type_lbl, img_key, start_)
    
    #do my stuff
    if len(train_data) < start_ + batch_s:
        print("{} Images have been processed.".format(total_iterations))
        break
    
    total_iterations +=batch_s
    start_ = end_
    end_ = end_ + batch_s   

In [ ]:
train_data, train_labels, img_type, img_keys = load_data(img_dir)

In [ ]:
train_orig = train_data[0, 0:1]
train_pred = train_data[1, 0:1]
labels = train_labels[0:1, 0]
labels
img_type_lbl = img_type[0:1]
img_key = img_keys[0:1]
dims = (1, num_classes, num_channels)
train, labels, img_type_lbl, img_key = get_batch_images(train_orig, train_pred, labels, img_type_lbl, img_key, dims, True)
# train[0:1]
# img_x = cv2.imread("../../original_images/SD/15970/0/img/predicted_mask.png")

# img_x = np.expand_dims(img_x[:,:,0].flatten(), axis=0)
np.set_printoptions(suppress=True)
# print(train)
output_cl1 = restore_see_layer(orig=np.expand_dims(train[0][0], axis=0), pred=np.expand_dims(train[1][0], axis=0), model_name=model2_50000,var_name='custom/custom_1_comb_patches')
# output_cl2 = restore_see_layer(ix=train,model_name=model2_50000,var_name='custom/custom_1_comb_patches')
print(output_cl1)
# show_img = np.expand_dims(np.reshape(output_cl1, (4,2)), axis=0)

In [ ]:
# print(output_cl1.shape)
new_img = np.reshape(output_cl1, [4, 2])
fig = plt.figure(figsize=(2,4))
plt.imshow(new_img, interpolation='none', aspect='auto')

In [ ]:
img_x = cv2.imread("../../original_images/SD/15970/0/labels/merged_patch.png")
print(img_x.shape)
reshaped_img_x = np.expand_dims(img_x, axis=0)

grey_img = np.dot(img_x[...,:3], [0.299, 0.587, 0.114])
print(grey_img.shape)

# grey_img = normalize(grey_img)

# see_output(reshaped_img_x, figsize=(2,4))

fig = plt.figure(figsize=(2,4))
# print(grey_img)
plt.imshow(grey_img, interpolation='none', aspect='auto')
# see_output(reshaped_img_x, figsize = (2,4))

In [ ]:
def get_tensors(graph=tf.get_default_graph()):
    return [t for op in graph.get_operations() for t in op.values()]
get_tensors()

In [ ]:
import pyodbc